In [1]:
# import sys
# sys.path.append('/mnt/disk1/yunseob/Pytorch/1_CapsuleEndo/')
from CE_utils import sm160_dataset, GenerateLabel
from CE_training import CNN_Training

import numpy as np
from itertools import product

ImportError: No module named 'CE_utils'

In [46]:
def sm160_load(tv_rate = 0.9):
    neg_train_aug_path, neg_train_x, neg_valid_x = sm160.Get_train_valid_set('n', 'neg', train_rate = tv_rate)
    redspot_train_aug_path, redspot_train_x, redspot_valid_x = sm160.Get_train_valid_set('h', 'redspot', train_rate = tv_rate)
    angio_train_aug_path, angio_train_x, angio_valid_x = sm160.Get_train_valid_set('h', 'angio', train_rate = tv_rate)
    active_train_aug_path, active_train_x, active_valid_x = sm160.Get_train_valid_set('h', 'active', train_rate = tv_rate)
    ero_train_aug_path, ero_train_x, ero_valid_x = sm160.Get_train_valid_set('d', 'ero', train_rate = tv_rate)
    ulcer_train_aug_path, ulcer_train_x, ulcer_valid_x = sm160.Get_train_valid_set('d', 'ulcer', train_rate = tv_rate)
    str_train_aug_path, str_train_x, str_valid_x = sm160.Get_train_valid_set('d', 'str', train_rate = tv_rate)

    # 실제 모델이 학습할 augmented training data의 경로를 클래스 기준에 따라 list로 정리  
    h_train_aug_path = np.hstack([redspot_train_aug_path, angio_train_aug_path, active_train_aug_path])
    d_train_aug_path = np.hstack([ero_train_aug_path, ulcer_train_aug_path, str_train_aug_path])
    a_train_aug_path = np.hstack([h_train_aug_path, d_train_aug_path])
    train_paths = [neg_train_aug_path, a_train_aug_path]

    # epcch마다 모델을 평가할 (not augmented) training set과 validation set 클래스 기준에 따라 정리
    h_train_X = np.vstack([redspot_train_x, angio_train_x, active_train_x])
    d_train_X = np.vstack([ero_train_x, ulcer_train_x, str_train_x])
    a_train_X = np.vstack([h_train_X, d_train_X])

    h_valid_X = np.vstack([redspot_valid_x, angio_valid_x, active_valid_x])
    d_valid_X = np.vstack([ero_valid_x, ulcer_valid_x, str_valid_x])
    a_valid_X = np.vstack([h_valid_X, d_valid_X])

    train_X = [neg_train_x, a_train_X]
    valid_X = [neg_valid_x, a_valid_X]
    
    return train_paths, train_X, valid_X

In [47]:
phase_a = [[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 0], [1, 1, 1]]
phase_a_label = ['---', 'f--', '-r-', '--b', 'fr-', 'frb']

In [48]:
sm160 = sm160_dataset(phase = 'train', data = 'sm_x160_v2', pre_a = phase_a[0], pre_b = False, ext_name = False)

# 1. custom dataloader (equal batch)

- 클래스마다 같은 크기의 배치를 들고와 stack한 후에 training batch를 구성
- 병변별로 imbalance 존재
- 기존에 cv2로 불러오기 때문에 오래 걸리는 거라는 생각..?

In [6]:
train_paths, train_X, valid_X = sm160_load()

negative - negative | augmented training data: 5619 |(For validation) train x: (5619, 3, 512, 512) valid x: (625, 3, 512, 512)
hemorrhagic - red_spot | augmented training data: 500 |(For validation) train x: (500, 3, 512, 512) valid x: (56, 3, 512, 512)
hemorrhagic - angioectasia | augmented training data: 16 |(For validation) train x: (16, 3, 512, 512) valid x: (2, 3, 512, 512)
hemorrhagic - active_bleeding | augmented training data: 385 |(For validation) train x: (385, 3, 512, 512) valid x: (43, 3, 512, 512)
depressed - erosion | augmented training data: 663 |(For validation) train x: (663, 3, 512, 512) valid x: (74, 3, 512, 512)
depressed - ulcer | augmented training data: 658 |(For validation) train x: (658, 3, 512, 512) valid x: (74, 3, 512, 512)
depressed - stricture | augmented training data: 92 |(For validation) train x: (92, 3, 512, 512) valid x: (11, 3, 512, 512)


In [7]:
def batch_idxs(dataset, batch_size = 32, shuffle = True):

    idxs = np.arange(len(dataset))
    total_size = len(idxs)
    if shuffle:
        np.random.shuffle(idxs)
    start = 0
    b_idxs = []
    while True:
        if total_size > start + batch_size: 
            b_idxs.append(list(idxs[start:start+batch_size]))  
            start += batch_size
        elif total_size <= start + batch_size: 
            b_idxs.append(list(idxs[start:]))
            break 
    return b_idxs

def Reshape4torch(img):
    """
    (sample #, height, width, channel) -> (sample #, channel, height, width)
    """
    img = np.transpose(img, (0, 3, 1, 2))
    return img
    
def GenerateLabel(data, cls):
#     label = cls*np.ones([data.shape[0]])
    label = cls*np.ones([len(data)])
    return label

def LoadRandomMinibatch(pathlist, cls, batch_size = 50, image_ch = 'bgr'):
    import cv2
    idx = np.random.choice(len(pathlist), batch_size)
    batch_dir = pathlist[idx]
    batch_x = []
    for i in batch_dir:
        temp = cv2.imread(i)
        if image_ch == 'bgr':
            pass
        elif image_ch == 'rgb':
            temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
        elif image_ch == 'hsv':
            temp = cv2.cvtColor(temp, cv2.COLOR_BGR2HSV)
        temp = temp / 255
        temp = (temp - 0.5) / 0.5
        batch_x.append(temp)
    batch_label = GenerateLabel(batch_x, cls)
    return Reshape4torch(np.asarray(batch_x)), batch_label

def Shuffle(x1, x2):
    """
    random shuffle of two paired data -> x, y = shuffle(x, y)
    but, available of one data -> x = shuffle(x, None)
    """
    idx = np.arange(len(x1))
    np.random.shuffle(idx)
    if type(x1) == type(x2):
        return x1[idx], x2[idx] 
    else:
        return x1[idx]
    
def Torch_Minibatch_Load(train_paths, batch_size = 100, shuffle = False):
    x, y = [], []
    for X, cls in zip(train_paths, range(len(train_paths))):
#             x_i, y_i = RandomMinibatch(X, Y, batch_size)
        x_i, y_i = LoadRandomMinibatch2(X, cls, batch_size)
        x.append(x_i), y.append(y_i)
    x, y = np.concatenate(x), np.concatenate(y)
    if shuffle != False:
        x, y = Shuffle(x, y)
    x, y = torch.tensor(x, device = device).float(), torch.tensor(y, device = device).long()
#         x, y = torch.tensor(x, device = 'cpu').float(), torch.tensor(y, device = 'cpu').long()
    return x, y

def Validation(X, Y, batch_size = 32):
    batch_idxs = Batch_idxs(X, batch_size)
    output = []
    for batch in batch_idxs:
        x = torch.tensor(X[batch, :, :, :], device = device).float() 
        o = model(x)
        output.append(o)
    output = torch.cat(output)
    loss = criterion(output, Y)

    _, pred = torch.max(output, 1)
    return loss, pred   

In [8]:
def LoadRandomMinibatch2(pathlist, cls, batch_size = 50, image_ch = 'bgr'):
    idx = np.random.choice(len(pathlist), batch_size)
    batch_dir = pathlist[idx]
    batch_x = []
    for i in batch_dir:
        temp = Image.open(i)
        temp = F.to_tensor(temp)
        temp = F.normalize(temp, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5],inplace=False)
        batch_x.append(temp)
    batch_label = GenerateLabel(batch_x, cls)
    return torch.stack(batch_x), batch_label

def Torch_Minibatch_Load2(train_paths, batch_size = 100, shuffle = False):
    x, y = [], []
    for X, cls in zip(train_paths, range(len(train_paths))):
        x_i, y_i = LoadRandomMinibatch2(X, cls, batch_size)
        
        x.append(x_i), y.append(y_i)

    x, y = torch.cat(x), torch.tensor(np.hstack(y))
    if shuffle != False:
        x, y = Shuffle(x, y)
    x, y = x.to(device = device), y.to(device = device)
#     x, y = torch.tensor(x, device = device).float(), torch.tensor(y, device = device).long()
#         x, y = torch.tensor(x, device = 'cpu').float(), torch.tensor(y, device = 'cpu').long()
    return x, y

In [9]:
import torch
from torchvision.transforms import functional as F
from PIL import Image

In [12]:
train_X = torch.tensor(np.concatenate(train_X))
train_X.shape

torch.Size([7933, 3, 512, 512])

In [15]:
valid_X = torch.tensor(np.concatenate(valid_X))
valid_X.shape

torch.Size([885, 3, 512, 512])

In [10]:
device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")

In [11]:
n_data = [5619, 0]

In [14]:
# cv2

n_epoch = 1
n_batch = 16
iter_i = 0
epoch_i = 0
patient_i = 0

while True:
    iter_i += 1
    train_x, train_y = Torch_Minibatch_Load(train_paths, n_batch, shuffle = True)

    if iter_i % (np.max(n_data) // n_batch + 1) == 0:   
        epoch_i += 1

    if epoch_i == n_epoch:
        break

In [15]:
# cv2
# validation까지

n_epoch = 1
n_batch = 16
iter_i = 0
epoch_i = 0
patient_i = 0

while True:
    iter_i += 1
#             train_x, train_y = Torch_Minibatch_Load(train_Xs, train_Ys, n_batch, shuffle = True)
    train_x, train_y = Torch_Minibatch_Load(train_paths, n_batch, shuffle = True)

#     model.train(); optimizer.zero_grad()

    if iter_i % (np.max(n_data) // n_batch + 1) == 0:   
        epoch_i += 1
        patient_i += 1

#         model.eval()
        with torch.no_grad():
            
#             valid_X = torch.tensor(np.concatenate(valid_X))
            v_idxs = batch_idxs(valid_X, 16)

            for v_idx in v_idxs:
                valid_x = valid_X[v_idx]

#             if iter_i == n_iter:
    if epoch_i == n_epoch:
        break

In [16]:
# cv2
# validation까지 (데이터증강 전 훈련데이터 포함)
# !! 현재 돌리고 있는 코드와 가장 유사함. 
# train_Y, valid_Y 안 넣은 거 빼고 

n_epoch = 1
n_batch = 16
iter_i = 0
epoch_i = 0
patient_i = 0

while True:
    iter_i += 1
#             train_x, train_y = Torch_Minibatch_Load(train_Xs, train_Ys, n_batch, shuffle = True)
    train_x, train_y = Torch_Minibatch_Load(train_paths, n_batch, shuffle = True)

#     model.train(); optimizer.zero_grad()

    if iter_i % (np.max(n_data) // n_batch + 1) == 0:   
        epoch_i += 1
        patient_i += 1

#         model.eval()
        with torch.no_grad():
            
    
            t_idxs = batch_idxs(train_X, 16)

            for t_idx in t_idxs:
                train_x = train_X[t_idx]
    
            v_idxs = batch_idxs(valid_X, 16)

            for v_idx in v_idxs:
                valid_x = valid_X[v_idx]

#             if iter_i == n_iter:
    if epoch_i == n_epoch:
        break

In [ ]:
valid_X

In [14]:
v_idx

[1, 0]

In [12]:
# transform에서 쓰는 함수 끌고와서 기존 cv2함수를 교체해봄

n_epoch = 1
n_batch = 16
iter_i = 0
epoch_i = 0
patient_i = 0

while True:
    iter_i += 1
    train_x, train_y = Torch_Minibatch_Load2(train_paths, n_batch, shuffle = True)

    if iter_i % (np.max(n_data) // n_batch + 1) == 0:   
        epoch_i += 1

    if epoch_i == n_epoch:
        break

In [13]:
# transform에서 쓰는 함수 끌고와서 기존 cv2함수를 교체해봄
# validation까지

n_epoch = 1
n_batch = 16
iter_i = 0
epoch_i = 0
patient_i = 0

while True:
    iter_i += 1
#             train_x, train_y = Torch_Minibatch_Load(train_Xs, train_Ys, n_batch, shuffle = True)
    train_x, train_y = Torch_Minibatch_Load2(train_paths, n_batch, shuffle = True)

#     model.train(); optimizer.zero_grad()

    if iter_i % (np.max(n_data) // n_batch + 1) == 0:   
        epoch_i += 1
        patient_i += 1

#         model.eval()
        with torch.no_grad():
            
#             valid_X = torch.tensor(np.concatenate(valid_X))
            v_idxs = batch_idxs(valid_X, 16)

            for v_idx in v_idxs:
                valid_x = valid_X[v_idx]

#             if iter_i == n_iter:
    if epoch_i == n_epoch:
        break

In [ ]:
# transform에서 쓰는 함수 끌고와서 기존 cv2함수를 교체해봄
# validation까지

n_epoch = 1
n_batch = 16
iter_i = 0
epoch_i = 0
patient_i = 0

while True:
    iter_i += 1
#             train_x, train_y = Torch_Minibatch_Load(train_Xs, train_Ys, n_batch, shuffle = True)
    train_x, train_y = Torch_Minibatch_Load2(train_paths, n_batch, shuffle = True)

#     model.train(); optimizer.zero_grad()

    if iter_i % (np.max(n_data) // n_batch + 1) == 0:   
        epoch_i += 1
        patient_i += 1

#         model.eval()
        with torch.no_grad():
            
#             valid_X = torch.tensor(np.concatenate(valid_X))
            v_idxs = batch_idxs(valid_X, 16)

            for v_idx in v_idxs:
                valid_x = valid_X[v_idx]

#             if iter_i == n_iter:
    if epoch_i == n_epoch:
        break

# 2. torch dataset 구성 

- class별로 normal과 abnormal을 거의 equal하게 맞추는데,
- 대신에 negative size와 지정한 병변의 갯수(6)에 따라 equal하게 negative의 1/N oversampling함 (strict)
- oversampling이기 때문에 위에서 랜덤으로 equal하게 뽑는 것보단 편향이 발생하기 더 쉬움
- batch loader 시간은 빠른 편

In [49]:
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from functools import partial

In [50]:
neg_path = sm160.load_path('n', 'neg')
redspot_path = sm160.load_path('h', 'redspot')
angio_path = sm160.load_path('h', 'angio')
active_path = sm160.load_path('h', 'active')
ero_path = sm160.load_path('d', 'ero')
ulcer_path = sm160.load_path('d', 'ulcer')
str_path = sm160.load_path('d', 'str')

print("negative:",neg_path.shape)
print("red spot:",redspot_path.shape)
print("angioectasia:",angio_path.shape)
print("active_bleeding:",active_path.shape)
print("erosion:",ero_path.shape)
print("ulcer:",ulcer_path.shape)
print("stricture:",str_path.shape)

negative: (6244,)
red spot: (556,)
angioectasia: (18,)
active_bleeding: (428,)
erosion: (737,)
ulcer: (732,)
stricture: (103,)


In [1]:
import os
from glob import glob
import pandas as pd

from PIL import Image

In [81]:
class CE_dataset(Dataset):
    def __init__(self, path, data_dir = 'sm_x160_v2',
                 class_config = {'n': ['neg'], 
                                 'a': {'h': ['redspot', 'angio', 'active'], 
                                       'd': ['ero', 'ulcer', 'str']}},
                 oversample=False, mode = 'binary', transform=None,
                 valid_separate=False, valid_df=None):
    
        self.class_config = class_config
        self.oversample = oversample
        self.mode = mode
        self.transform = transform
        self.data_dir = os.path.join(path[0].split(data_dir)[0], data_dir)
        
        self.lesions = dict(neg = 'negative', 
                       redspot = 'red_spot', angio = 'angioectasia', active = 'active_bleeding', 
                       ero = 'erosion', ulcer = 'ulcer', str = 'stricture', 
                       amp = 'ampulla_of_vater', lym = 'lymphoid_follicles', tum = 'small_bowel_tumor')
        self.types = dict(n = 'negative', h = 'hemorrhagic', d = 'depressed', p = 'protruded')
        
        self.types_cols = ([self.types['n']]*len(self.class_config['n']) + 
                           list(np.concatenate([[self.types[i]]*
                                                len(self.class_config['a'][i]) 
                                                for i in self.class_config['a'].keys()])))
        self.lesion_cols = ([self.lesions[i] for i in self.class_config['n']] + 
                       [self.lesions[i] for i in list(np.concatenate(
                           [i for i in self.class_config['a'].values()]))])
#         self.lesion_cols = ['negative', 
#                        'red_spot', 'angioectasia', 'active_bleeding', 
#                        'erosion', 'ulcer', 'stricture',
#                        'ampulla_of_vater', 'lymphoid_follicles', 'small_bowel_tumor']

        if valid_separate:
            self.subpath = [subpath.split(data_dir)[-1][1:] for subpath in path]
            
            self.df = pd.DataFrame(columns = [self.types_cols,
                                              self.lesion_cols])
            self.df_config()
            self.df, self.valid_df = self.df_train_valid_split()
            
        elif not valid_separate and valid_df is not None:
            self.df = valid_df
            
        if self.oversample:
            self.df_os = self.oversampling()

    def df_config(self): 
        names = []
        splits = []
        for i, path in enumerate(self.subpath):
            split, category1, category2, name = path.split('/')
            self.df.loc[i, (category1, category2)] = 1
            names.append(name), splits.append(split) 

        self.df.insert(0, 'name', names)
        self.df['split'] = splits
        self.df.fillna(0, inplace = True)
        
    def df_train_valid_split(self):
        train_df = []
        valid_df = []

        for i in self.class_config.keys():
            for j in self.class_config[i]:
                if i == 'a':
                    for k in self.class_config[i][j]:
                        a_df = self.df[self.df[self.types[j], self.lesions[k]] == 1] 
                        valid_idx = np.random.choice(len(a_df), int(0.2*len(a_df)), replace = False)
                        train_idx = np.setdiff1d(np.arange(len(a_df)), valid_idx)
                        valid_df.append(a_df.iloc[valid_idx])
                        train_df.append(a_df.iloc[train_idx])
                else:
                    n_df = self.df[self.df[self.types[i], self.lesions[j]] == 1] 
                    valid_idx = np.random.choice(len(n_df), int(0.2*len(n_df)), replace = False)
                    train_idx = np.setdiff1d(np.arange(len(n_df)), valid_idx)
                    valid_df.append(n_df.iloc[valid_idx])
                    train_df.append(n_df.iloc[train_idx])

        train_df = pd.concat(train_df)
        valid_df = pd.concat(valid_df)
        
        return train_df, valid_df

    def oversampling(self):
        n_df = self.df[self.df[self.types['n'], self.lesions[self.class_config['n'][0]]] == 1]
        n_max = len(n_df)

        n_a = 0
        for les in self.class_config['a'].values():
            n_a += len(les)

        n_ref = int(np.max(n_max) / (n_a))

        new_a_dfs = []
        for j in self.class_config['a']:
            for k in self.class_config['a'][j]:
                cls, les = self.types[j], self.lesions[k]
                a_df = self.df[self.df[cls, les] == 1]

                quo = n_ref//len(a_df)
                rem = n_ref%(len(a_df))
                rest_idx  = np.random.choice(len(a_df), rem, replace = False)

                a_df_os = pd.concat([a_df]*quo)
                a_df_os = a_df_os.append(a_df.iloc[rest_idx])
                new_a_dfs.append(a_df_os)
        new_a_df = pd.concat(new_a_dfs)
        new_df = new_a_df.append(n_df)

        return new_df

    def __getitem__(self, index):
#         if self.oversample:
#             self.df_os = self.oversampling()
        if self.oversample:
            row = self.df_os.iloc[index]
        else:
            row = self.df.iloc[index]
        sp, n = row['split'].tolist()[0], row['name'].tolist()[0]
        c1, c2 = row[row == 1].index.values[0]

        X = Image.open(os.path.join(self.data_dir, sp, c1, c2, n))

        target = []

        if self.mode == 'binary':
            if row['negative']['negative'] == 1:
                target.append(0)
            else:
                target.append(1)

        if self.transform is not None:
            X = self.transform(X)

        if target:
            target = tuple(target) if len(target) > 1 else target[0]

#             if self.target_transform is not None:
#                 target = self.target_transform(target)
        else:
            target = None

        return X, target
    
    def __len__(self):
        if self.oversample:
            self.df_os = self.oversampling()
            return len(self.df_os)
        elif not self.oversample:
            return len(self.df)

In [52]:
train_path = list(np.hstack([neg_path, 
                              redspot_path, angio_path, active_path, 
                              ero_path, ulcer_path, str_path]))

transformer = transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                 ])

class_config = {'n': ['neg'], 
                'a': {'h': ['redspot', 'angio', 'active'], 
                      'd': ['ero', 'ulcer', 'str']}}

training_dataset = CE_dataset(train_path, data_dir = 'sm_x160_v2',
                             class_config = class_config,
                              oversample=True, mode = 'binary', transform=transformer,
                              valid_separate=True)

validation_dataset = CE_dataset(train_path, data_dir = 'sm_x160_v2',
                             class_config = class_config,
                              oversample=False, mode = 'binary', transform=transformer,
                              valid_separate=False, valid_df=training_dataset.valid_df)

In [21]:
training_loader = torch.utils.data.DataLoader(dataset=training_dataset, batch_size=32, shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=32, shuffle=True)

In [22]:
x, y = training_loader.__iter__().__next__()

In [23]:
x.shape

torch.Size([32, 3, 512, 512])

In [24]:
y

tensor([1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 1, 1, 0, 1])

In [25]:
len(training_dataset)

9988

In [26]:
len(training_loader)

313

In [27]:
# oversampling 위로 뺐을때 ..

for i, (inputs, outputs) in enumerate(training_loader):
    x, y = inputs.to(device), outputs.to(device)

In [28]:
# oversampling 위로 뺐을때 ..
# vallidation 까지

for i, (inputs, outputs) in enumerate(training_loader):
    x, y = inputs.to(device), outputs.to(device)

else:
    with torch.no_grad():
        for i, (inputs, outputs) in enumerate(validation_loader):
            x, y = inputs.to(device), outputs.to(device)

# 3. Conclusion

- 의외로 속도면에서 차이가 별로 안 나..? (cv2로 짜놓은 것도 그렇게 구리진 않았다)
- 지금 속도가 걸리는 건 validation을 데이터증강전의 train_X까지 해서 ...? (아니네)
- Lr 3개 Bs 3개 9개 조합 뒤로 갈수록 데이터 많아지니 오래 걸리는 듯 (이게며칠째야)
- 병변별로도 strict하게 나누는 것이 과연 모델 성능에 영향이 있을 것인지 의문